<a href="https://colab.research.google.com/github/souradip93/CS69002_9A_18CS60R07/blob/master/Assignment9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import nltk
import re

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Test_5K.txt to Test_5K.txt


In [0]:
df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.txt'].decode('utf-8')), sep='\t')
df_train.head()

In [0]:
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.txt'].decode('utf-8')), sep='\t')
df_test.head()

In [6]:
print('Number of Negative movie reviews', len(df_train[df_train['label']==0]))
print('Number of Positive movie reviews', len(df_train[df_train['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


In [0]:
def remove_br_tags_and_numbers(train_data_X):
  import re
  regex = re.compile(r'<br.*?>|\d+')
  train_data_X = [regex.sub('', x) for x in train_data_X]
  return train_data_X

In [0]:

def tokenize(train_data_X):
  from nltk.tokenize import RegexpTokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  train_data_X = [tokenizer.tokenize(x) for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X


In [0]:
def convert_to_lower(train_data_X):
  train_data_X = [[y.lower() for y in x] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

In [0]:
def remove_stopwords(train_data_X):
  from nltk.corpus import stopwords
  nltk.download('stopwords')
  stopword_set = set(stopwords.words('english'))
  train_data_X = [[y for y in x if y not in stopword_set] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

In [0]:
def lemmatize(train_data_X):
  from nltk.stem import WordNetLemmatizer 
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  train_data_X = [[lemmatizer.lemmatize(y) for y in x] for x in train_data_X]
  return train_data_X

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  for sent in dataset:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
              
  word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_to_ix

In [0]:
train_data_X = df_train['text'].astype(str).tolist()
train_data_Y = df_train['label'].astype(int)

In [0]:
test_data_X = df_test['text'].astype(str).tolist()
test_data_Y = df_test['label'].astype(int)

In [0]:
train_data_X = remove_br_tags_and_numbers(train_data_X)
train_data_X = tokenize(train_data_X)
train_data_X = convert_to_lower(train_data_X)
train_data_X = lemmatize(train_data_X)

In [46]:
test_data_X = remove_br_tags_and_numbers(test_data_X)
test_data_X = tokenize(test_data_X)
test_data_X = convert_to_lower(test_data_X)
test_data_X = lemmatize(test_data_X)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
word_to_ix = generate_word_ids(train_data_X)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
print(VOCAB_SIZE, word_to_ix)

58320 {'john': 0, 'water': 1, 'ha': 2, 'given': 3, 'u': 4, 'a': 5, 'genuinely': 6, 'enjoyable': 7, 'film': 8, 'this': 9, 'certainly': 10, 'isn': 11, 't': 12, 'without': 13, 'it': 14, 'shocking': 15, 'esque': 16, 'moment': 17, 'but': 18, 'is': 19, 'tamer': 20, 'than': 21, 'his': 22, 'older': 23, 'culty': 24, 'stuff': 25, 'such': 26, 'pink': 27, 'flamingo': 28, 'pecker': 29, 'harkens': 30, 'back': 31, 'to': 32, 's': 33, 'early': 34, 'mainstream': 35, 'stage': 36, 'in': 37, 'that': 38, 'reminds': 39, 'the': 40, 'viewer': 41, 'of': 42, 'same': 43, 'kind': 44, 'humor': 45, 'wa': 46, 'evident': 47, 'polyester': 48, 'overall': 49, 'really': 50, 'fun': 51, 'comedy': 52, 'with': 53, 'some': 54, 'great': 55, 'first': 56, 'two': 57, 'season': 58, 'series': 59, 'were': 60, 'very': 61, 'strange': 62, 'and': 63, 'they': 64, 'weren': 65, 'funny': 66, 'had': 67, 'drama': 68, 'element': 69, 'where': 70, 'bill': 71, 'mother': 72, 'struggling': 73, 'all': 74, 'usual': 75, 'problem': 76, 'life': 77, 'bit'

In [26]:
label_to_ix = {1: 1, 0: 0}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 0, 1: 1}, {0: 0, 1: 1})

In [27]:
import torch.nn as nn
import torch.nn.functional as F

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.device('cuda')



device(type='cuda')

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin1 = nn.Linear(vocab_size, 50)
    self.lin2 = nn.Linear(50, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    return F.softmax(self.lin2(out))

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[ word_to_ix['UNKNOWN'] ] += 1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    #print(label)
    return torch.LongTensor([label_to_ix[label]])

In [0]:
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [0]:
from torch.autograd import Variable
import time

bow_vec = []
for instance in train_data_X:
  ins_bow_vector = make_bow_vector(instance, word_to_ix)
  bow_vec.append(Variable(ins_bow_vector).cuda())
  
label = []
for l in train_data_Y:
  label_vector = make_target(l, label_to_ix)
  label.append(Variable(label_vector).cuda())



In [43]:
# the training loop
for epoch in range(20):
#     print(epoch)
    start = time.time()
    for ind, instance in enumerate(train_data_X):
        # get the training data
        #label = train_data_Y[ind]
        bow.zero_grad()
        #bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        #label = Variable(make_target(label, label_to_ix)).cuda()
        #if ind==0:
        #  print(label) 
        probs = bow(bow_vec[ind]) # forward pass
        loss = loss_function(probs, label[ind])
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    end = time.time()
    print('Time Taken - ' + str(end - start))
    print('Epoch - ' + str(epoch) + ' , LOSS - ' + str(loss.data))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 25.86399531364441
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.096599817276
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.666008710861206
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.05334734916687
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.605652570724487
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.047956466674805
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.53832483291626
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.038834810256958
Epoch - 7 , LOSS - tensor(0., device='cuda:0')
Time Taken - 25.064181089401245
Epoch - 8 , LOSS - tensor(-3.2891e-14, device='cuda:0')
Time Taken - 25.605757474899292
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.087680101394653
Epoch - 10 , LOSS - tensor(0., device='cuda:0')
Time Taken - 25.061164617538452
Epoch - 11 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 25.097236394882202
Epoch 

In [44]:
torch.save(bow,'model.json')

from google.colab import files
files.download("model.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
from google.colab import files
temp_test = files.upload()


In [0]:
torch.load(io.BytesIO(temp_test['model.json']))

In [47]:
print('--- AFTER TRAINING ---')
for ind, instance in enumerate(test_data_X):
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = bow(bow_vec)
    print(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    print('prediction: {}'.format(ix_to_label[pred]))
    label = test_data_Y[ind]
    print('actual: {}'.format(label))

--- AFTER TRAINING ---


RuntimeError: ignored